In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/miraeasset'

/content/drive/MyDrive/miraeasset


In [ ]:
%pwd

'/content/drive/MyDrive/miraeasset'

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

In [ ]:
# 최종 결과를 저장할 빈 데이터프레임 생성
real_result = pd.DataFrame()

# 누락된 콘텐츠와 답변을 위한 오류 카운트 초기화
content_err = 0
answer_err = 0

# 데이터 검색 연도 설정
year = 2016

# 1월(1)부터 12월(12)까지 월을 순환
for month in range(1, 13):
    # 1부터 100까지 페이지 번호를 순환
    for page_num in range(1, 101):
        # 현재 페이지 데이터를 저장할 데이터프레임 초기화
        result = pd.DataFrame(columns=['question', 'answer', 'date'])

        # 현재 월과 년도를 사용하여 페이지 링크 생성
        link = f'https://kin.naver.com/search/list.nhn?sort=none&query=%26quot%3B%EA%B3%B5%EB%AA%A8%EC%A3%BC%26quot%3B&period={year}.0{month}.01.%7C{year}.0{month}.31.&section=kin&page={page_num}'
        print('-' * 30)
        print("pagenum:", page_num)
        link_list = []

        # 생성한 링크로 HTTP GET 요청 보내기
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        c = soup.find_all('dt')

        # 현재 월에 질문이 없는 경우 처리
        if len(c) == 0:
            print(f'************************{month}월에 대한 검색 결과 끝************************')
            break

        # 페이지에서 질문 링크 추출
        for i in range(len(c)):
            temp = c[i]
            temp2 = temp.find('a', {'class': '_nclicks:kin.txt _searchListTitleAnchor'})['href']
            link_list.append(temp2)

        # 질문과 답변 데이터를 저장할 데이터프레임 초기화
        df = pd.DataFrame(columns=['question', 'answer', 'date'])

        # 각 질문 링크 처리
        for i in range(len(link_list)):
            answer_list = []
            link = link_list[i]
            response = requests.get(link)
            soup = BeautifulSoup(response.text, 'html.parser')

            # 질문의 날짜 추출
            date = soup.find('span', {'class': 'c-userinfo__info'}).text[3:]

            # 질문 콘텐츠 추출
            content = soup.find('div', {'class': 'c-heading__content'})
            if content is None:
                try:
                    content = soup.find('div', {'class': 'title'}).text
                except:
                    content = ''
                    content_err += 1
                    print('**content is NONE**')
            else:
                content = content.text
            print(i, '번째질문:', content)

            # 질문에 대한 답변 추출
            temp = soup.find_all('div', {'class': 'answer-content__item _contentWrap _answer'})
            for j in range(len(temp)):
                a = temp[j]
                answer = a.find('div', {'class': 'se-module se-module-text'})
                if answer is None:
                    try:
                        answer = a.find('div', {'class': '_endContentsText c-heading-answer__content-user'}).text
                    except:
                        answer = ''
                        answer_err += 1
                        print('**answer is NONE**')
                else:
                    answer = answer.text
                answer_list.append(answer)

            # 질문, 답변 및 날짜를 데이터프레임에 추가
            df = df.append({'question': content, 'answer': answer_list, 'date': date}, ignore_index=True)
        print("df 몇개?", len(df), "개")

        # 현재 페이지의 데이터프레임을 결과 데이터프레임과 병합
        result = pd.concat([result, df], ignore_index=True)
        print("result 몇개?", len(result), "개")

        # 현재 페이지의 결과 데이터프레임을 최종 결과 데이터프레임과 병합
        real_result = pd.concat([real_result, result], ignore_index=True)

        # 이 페이지의 질문 수가 10개 미만인 경우, 해당 월 검색 결과 종료
        if len(c) < 10:
            print(f'************************{month}월에 대한 검색 결과 끝************************')
            break

# 결과 출력 및 CSV 파일로 저장
print("*" * 30)
print("real_result 몇개?", len(real_result), "개")
print("content 누락 :", content_err)
print("answer 누락 :", answer_err)
real_result.to_csv('지식in_{}.csv'.format(year))
print("지식in_{}.csv 저장완료!".format(year))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2 번째질문: 
						이번 skeit 공모주 청약하는데 이게 처음이라 궁금한게 있어서 질문드립니다현재 1주 10만5천원 확정이고 10주 청약하는데 필요한 금액이 5주분량인 52만5천원 이라 하더라구요청약이 처음이라 이게 이해가 안되는데 왜 10주 청약에 5주분의 금액만 필요한지 궁금합니다또 이 증거금의 개념도 설명해주시면 감사하겠습니다좋은 하루 되세요~!
					
3 번째질문: 
						그리고 주식수가 많은 곳에 신청하는 것이 유리한가요? 많은 곳은 증권 계좌가 없고 적은 곳은 있어서요.
					
4 번째질문: 
						공모주 청약은 주간사에서만 거래가 가능한가요?주간사에 키움증권이 없으면 키움증권으로는 거래를 못하는 건가요?
					
5 번째질문: 
						안녕하세요! 최근 주식시장의 변동성이 너무 심해 저같은 초보투자자에게 수익을 내기가 참 어렵네요. 그래서, 유망한 신규 상장 공모주에 관심을 갖게 됐습니다. 공모주 일반 청약과 관련해 몇 가지 질문드리겠습니다. 1. 인기 종목은 경쟁률이 1000 대 1도 넘는 경우가 있던데, 이러한 경쟁률을 뚫고 공모주를 우선 배정받을 수 있는 방법이 있을까요? 2. 신청 주식수 대비 청약금은 얼마를 납입해야 하나요? 3. 공모주를 신청하고 청약금을  완납해도 청약 당첨이 안되거나  청약 주식수의 일부만 당첨되는 경우도 있나요? 4. 청약 당첨에 실패할 경우 납입금 전액 또는 미당첨분에 해당되는 금액을 바로 돌려 받을 수 있나요? 

6 번째질문: 
						어떤 기업이 상장한다 해서 공모주 청약을 진행합니다 공모가 가격이 만약 3000원 이다 해서 100주를 신청해 100주가 청약되면 300백만원 정도 가지고 있는건데 먄약 상장일에 가격이 떨어질수 있는지 아니면 공모가로 샀는데 마이너스가 될수있나요?아니면 최종 상장 가격이 변동되어 마이너스를 볼수 있나요?

7 번째질문: 
						ISA로 새빗켐 공모주 넣었는데 출금이 35만원 